# pyoctree introduction
---
Requirements:
* pyoctree
* vtk >= 6.2.0

In [1]:
# Imports
from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys, vtk
sys.path.append('../')
import pyoctree
from pyoctree import pyoctree as ot
import time

In [2]:
print('pyoctree version = ', pyoctree.__version__)
print('vtk version = ', vtk.vtkVersion.GetVTKVersion())

pyoctree version =  0.2.4
vtk version =  8.1.0


## Load 3D model geometry (stl file)

In [3]:
# Read in stl file using vtk
reader = vtk.vtkSTLReader()
reader.SetFileName("knot.stl")
reader.MergingOn()
reader.Update()
stl = reader.GetOutput()
print("Number of points    = %d" % stl.GetNumberOfPoints())
print("Number of triangles = %d" % stl.GetNumberOfCells())

Number of points    = 38214
Number of triangles = 76428


In [4]:
# Extract polygon info from stl

# 1. Get array of point coordinates
numPoints   = stl.GetNumberOfPoints()
pointCoords = np.zeros((numPoints,3),dtype=float)
for i in range(numPoints):
    pointCoords[i,:] = stl.GetPoint(i)
    
# 2. Get polygon connectivity
numPolys     = stl.GetNumberOfCells()
connectivity = np.zeros((numPolys,3),dtype=np.int32)
for i in range(numPolys):
    atri = stl.GetCell(i)
    ids = atri.GetPointIds()
    for j in range(3):
        connectivity[i,j] = ids.GetId(j)

In [5]:
# Show format of pointCoords
pointCoords

array([[ 2.54317427,  7.41368246,  2.8599999 ],
       [ 2.51169538,  7.40434027,  2.8599999 ],
       [ 2.48648334,  7.48027468,  2.8599999 ],
       ..., 
       [ 1.20494771,  8.17073154,  2.8599999 ],
       [ 1.20671427,  8.17605209,  2.8599999 ],
       [ 1.20778596,  8.18155479,  2.8599999 ]])

In [6]:
# Show format of connectivity
connectivity

array([[    0,     1,     2],
       [    2,     3,     0],
       [    3,     4,     0],
       ..., 
       [38182, 38181, 38180],
       [38190, 38189, 38187],
       [38189, 38188, 38187]], dtype=int32)

## Generate octree

In [7]:
# Create octree structure containing stl poly mesh
max_depth = 5
#tree = ot.PyOctree(pointCoords,connectivity, int max_point_per_voxel, int max_octree_depht)
t=time.time()
tree = ot.PyOctree(pointCoords,connectivity,100,max_depth)
print(time.time()-t)

0.9512801170349121


In [8]:
# Print out basic Octree data
print("Size of Octree               = %.3fmm" % tree.root.size)
print("Number of Octnodes in Octree = %d" % tree.getNumberOfNodes())
print("Number of polys in Octree    = %d" % tree.numPolys)

Size of Octree               = 3.115mm
Number of Octnodes in Octree = 1753
Number of polys in Octree    = 76428


# Separate inside cells from the outside one

In [9]:
print(tree.getNumberOfNodes())
print(tree.getNumberOfLeafs())
        

1753
1534


In [10]:
a=tree.getLeafs()
len(a)
n=0
for i in a:
    if i.isLeaf:
        n+=1
print(n)

1534


In [17]:
b = tree.getNodes()
print(len(b))
for i in range(20):
    print(b[i])

1753
<PyOctnode, Id: 0, isLeaf: False, numPolys: 0>
<PyOctnode, Id: 0-0, isLeaf: False, numPolys: 0>
<PyOctnode, Id: 0-0-0, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-1, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-2, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3, isLeaf: False, numPolys: 0>
<PyOctnode, Id: 0-0-3-0, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3-1, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3-2, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3-3, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3-4, isLeaf: True, numPolys: 24>
<PyOctnode, Id: 0-0-3-5, isLeaf: False, numPolys: 0>
<PyOctnode, Id: 0-0-3-5-0, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3-5-1, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3-5-2, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3-5-3, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3-5-4, isLeaf: True, numPolys: 195>
<PyOctnode, Id: 0-0-3-5-5, isLeaf: True, numPolys: 0>
<PyOctnode, Id: 0-0-3-5-6, isLeaf: True, numPolys: 0>
<PyO

## Explore tree nodes

In [ ]:
# Get the root node
print(tree.root)

In [ ]:
# Get the branches of the root node, OctNode 0
tree.root.branches

In [ ]:
# Get OctNode 0-0 (first branch of the root node)
print(tree.root.branches[0])

In [ ]:
# Get the branches of OctNode 0-0 (first branch of the root node) using getNodeFromId function
for i in range(8):
    v_id = '0-0-'+str(i)
    print(tree.getNodeFromId(v_id).level)
print(tree.root.position)
print(tree.root.size)

In [ ]:
# An octnode that has no branches is a leaf
print('OctNode 0-0-1 is a leaf = ', tree.getNodeFromId('0-3-1').isLeaf)
print(tree.getNodeFromId('0-3-1'))

## Explore polygons in tree

In [ ]:
# Get list of first 10 polygons stored in tree
tree.polyList[0:10]

In [ ]:
# Get details of the first tri in the tree
tri = tree.polyList[0]
s = []
for i in range(3):
    s.append('[%.3f, %.3f, %.3f]' % tuple(tri.vertices[i][:3]))
s = ','.join(s)

print('Tri label = %d' % tri.label)
print('Vertex coordinates = %s' % s)
print('Face normal direction = [%.2f, %.2f, %.2f]' % tuple(tri.N))
print('Perp. distance from origin = %.3f' % tri.D)

In [ ]:
# Find the OctNode that the tri with label=0 lies within
tree.getNodesFromLabel(0)

In [ ]:
# Test if OctNode contains a particular tri label
tree.getNodeFromId('0-3-1-4-5-4').hasPolyLabel(0)

In [ ]:
#tree.getNodeFromId('0-3-1-4-5-4').get_data2()

## Find intersections between 3D object and ray

In [ ]:
# Create a list containing a single ray
xs,xe,ys,ye,zs,ze = stl.GetBounds()
x = 0.5*np.mean([xs,xe])
y = np.mean([ys,ye])
rayPointList = np.array([[[x,y,zs],[x,y,ze]]],dtype=np.float32)

In [ ]:
# Find if an intersection occurred
for i in tree.rayIntersections(rayPointList):
    print(i==1)

In [ ]:
# Get intersection points for a single ray
ray = rayPointList[0]
for i in tree.rayIntersection(ray):
    print('Intersection coords = [%.2f, %.2f, %.2f]' % tuple(i.p), ',  Parametric. dist. along ray = %.2f' % i.s)

## Write out a vtk file of the octree structure 

In [ ]:
# Create a vtk representation of Octree
tree.getOctreeRep()